## ** The Improved Power Disaggregation (Non-intrusive Load Monitoring) **

- Author : Gyubaek Kim (database.kim@yonsei.ac.kr)
- Description
  * Various algorithms have been utilized by disaggregation systems. The existing methods adopted neural network-based algorithms. The neural 
  network-based algorithms performed better than combinational optimization and hidden Markov models. Thus, we compared the 
  performance of the neural network-based algorithms and decision tree-based ensemble algorithms, such as random forest and gradient 
  boosting machine (GBM).
- License
   * All rights reserved. No part of this code may be reproduced or transmitted in any form or by any means, or utilized by any information storage and retrieval system without written permission from the copyright owner. You can use this source code for free projects only. I will not allow commercial use of this code. I am not allowing anyone to modify any or all parts of the source code for commercial purposes. By using my source code, you agree to the following: 
       * 1. You will not distribute any or all parts of this source code for commercial use 
       * 2. You will cite me as the original creator of this source code. 
       * 3. You will inform me of its use before you use any or all parts of this source code. 
   For use of any or all parts of this source code, email me. This source code is provided free to use for everybody provided this is used non-commercially. Violators of this agreement will be subject to legal action by the author. If you see commercial applications or software using any or all parts of this source code, inform me.  

#### import packages of benchmark project (contrib. project) to be compared

- https://github.com/OdysseasKr/neural-disaggregator
- The project implemented 5 neural network-based NILM models
  * RNN (Recurrent Neural Network)
  * GRU (Gated Recurrent Unit)
  * Window GRU
  * DAE (Denosing Autoencoder)
  * S2P (Short Sequence to Point)
  
- https://github.com/rgtzths/ICMLC_ICWAPR_code_base
- The project implemented ResNet-based NILM model
  * ResNet (Residual Network)

In [1]:
#### NIMLTK as common framework ####
from nilmtk.utils import find_nearest
from nilmtk.feature_detectors import cluster
from nilmtk.legacy.disaggregate import Disaggregator
from nilmtk.datastore import HDFDataStore
from nilmtk import DataSet

#### Benchmark Resource ####
from benchmark.rnndisaggregator import RNNDisaggregator
from benchmark.grudisaggregator import GRUDisaggregator
from benchmark.windowgrudisaggregator import WindowGRUDisaggregator
from benchmark.daedisaggregator import DAEDisaggregator
from benchmark.shortseq2pointdisaggregator import ShortSeq2PointDisaggregator
from benchmark.metrics import recall_precision_accuracy_f1, relative_error_total_energy, mean_absolute_error

#### Additional Benchmark Resource ####
from benchmark.resnet import RESNETDisaggregator

from work.benchmark.rnndisaggregator_fe import RNNDisaggregatorFE
from work.benchmark.grudisaggregator_fe import GRUDisaggregatorFE
from work.benchmark.windowgrudisaggregator_fe import WindowGRUDisaggregatorFE
from work.benchmark.daedisaggregator_fe import DAEDisaggregatorFE
from work.benchmark.shortseq2pointdisaggregator_fe import ShortSeq2PointDisaggregatorFE

#### Our Model ####
from work.dtdisaggregator import DTDisaggregator
from work.utils import checkSimilarity

import pandas as pd
import numpy as np
import h5py
import numpy.random
import time
import datetime
import math
import itertools
import warnings

from scipy.signal import find_peaks

from matplotlib import rcParams
import matplotlib.pyplot as plt

numpy.random.seed(42)
warnings.filterwarnings('ignore')

%matplotlib inline
rcParams['figure.figsize'] = (23, 10)

/home/kgbmanse/anaconda3/envs/nilm/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


#### Configuration

In [2]:
###############################################################
# Common
###############################################################
TRAIN_SAMPLE_PERIOD = 1
TEST_SAMPLE_PERIOD = 1

EPOCHS = 1

DATA_FILE = 'data/enertalk.h5'

T_S = '2017-01-01'
T_E = '2017-01-20'
P_S = '2017-01-21'
P_E = '2017-01-31'

HOME_ID = 1
appliances = ['television', 'washing machine', 'rice cooker', 'water filter', 'microwave']
#appliances = ['rice cooker', 'water filter', 'microwave']
#appliances = ['rice cooker']

In [3]:
# start_time, end_time : YYYY-MM-DD HH:MI:SS
def disaggregate(dataset, model, appliance, start_time, end_time, evaluate=False):
    
    dataset.set_window(start=start_time, end=end_time)
    
    test_elec = dataset.buildings[HOME_ID].elec    
    test_mains = test_elec.all_meters()[0]   
    test_meter = test_elec.submeters()[appliance]
    
    tempfile = 'work/temp/' + str(time.time())
    
    output = HDFDataStore(tempfile, 'w')    
    model.disaggregate(test_mains, output, test_meter, sample_period=TEST_SAMPLE_PERIOD)
    output.close()
    
    result = DataSet(tempfile)    
    res_elec = result.buildings[HOME_ID].elec

    predicted = res_elec[appliance] 
    
    ground_truth = None
   
    if evaluate == True:
        
        ground_truth = test_elec[appliance]

        # Performance Metric
        print("============ Relative error in total energy: {}".format(relative_error_total_energy(predicted, ground_truth)))
        print("============ Mean absolute error(in Watts): {}".format(mean_absolute_error(predicted, ground_truth)))
        
        # Performance Metric
        rpaf = recall_precision_accuracy_f1(predicted, ground_truth)
        print("============ Recall: {}".format(rpaf[0]))
        print("============ Precision: {}".format(rpaf[1]))
        print("============ Accuracy: {}".format(rpaf[2]))
        print("============ F1 Score: {}".format(rpaf[3]))
        
    return predicted

## Alogorithm Comparison

- Description
  In the first experiemnt, only total electricity consumption was used as training data

In [4]:
dataset = DataSet(DATA_FILE)
dataset.set_window(start=T_S, end=T_E)

train_elec = dataset.buildings[HOME_ID].elec
train_mains = train_elec.all_meters()[0] # The aggregated meter that provides the input

for aname in appliances:
    
    train_meter = train_elec.submeters()[aname]
    
    models = {}
    
    models['GRU-' + aname] = GRUDisaggregator()
    models['RNN-' + aname] = RNNDisaggregator()
    models['WGRU-' + aname] = WindowGRUDisaggregator(window_size=50)
    models['DAE-' + aname] = DAEDisaggregator(256)
    models['S2P-' + aname] = ShortSeq2PointDisaggregator(window_size=100)
    models['DRF-' + aname] = DTDisaggregator(algorithm='DRF', use_feature=False, reactive=False)
    models['GBM-' + aname] = DTDisaggregator(algorithm='GBM', use_feature=False, reactive=False)
    models['RESNET-' + aname] = RESNETDisaggregator()
    
    for mname, model in models.items():
        
        try:
            
            print("*"*20)
            print("[" + mname + "]") 
            print("*" *20)

            start = time.time()
            model.train(train_mains, train_meter, epochs=EPOCHS, sample_period=TRAIN_SAMPLE_PERIOD)
            end = time.time()            
            print("elapsed time (secs) = " + str(end-start))
            
            disaggregate(dataset, model, aname, P_S, P_E, True)
        
        except Exception as e:
            print('train & test error', e)


2022-07-01 13:25:14.617487: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory
2022-07-01 13:25:14.619178: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2022-07-01 13:25:14.619232: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1835] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2022-07-01 13:25:14.620473: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Libra

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_3 (Conv1D)            (None, 50, 16)            80        
_________________________________________________________________
bidirectional_4 (Bidirection (None, 50, 128)           31488     
_________________________________________________________________
dropout (Dropout)            (None, 50, 128)           0         
_________________________________________________________________
bidirectional_5 (Bidirection (None, 256)               198144    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 128)               32896     
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)              

2022-07-01 13:26:10.049959: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


12571/12571 [==============================] - 161s 12ms/step - loss: 8.3385e-05
elapsed time (secs) = 213.24771451950073
New sensible chunk: 357621
New sensible chunk: 173735
New sensible chunk: 332127
============ Relative error in total energy: 0.1760326745768233
============ Mean absolute error(in Watts): 31.43260061551896
============ Recall: 0.946935706318179
============ Precision: 0.42402476681374623
============ Accuracy: 0.5647593867149364
============ F1 Score: 0.585756044653722
********************
[RNN-television]
********************
6750/6750 [==============================] - 112s 16ms/step - loss: 1.1177e-04
elapsed time (secs) = 134.64802622795105
New sensible chunk: 357621
New sensible chunk: 173735
New sensible chunk: 332127
============ Relative error in total energy: 0.04027350796761409
============ Mean absolute error(in Watts): 32.16830288464859
============ Recall: 0.9770316069322138
============ Precision: 0.38387371392406483
============ Accuracy: 0.482935930

[Parallel(n_jobs=32)]: Using backend ThreadingBackend with 32 concurrent workers.


...

New sensible chunk: 173735


[Parallel(n_jobs=32)]: Using backend ThreadingBackend with 32 concurrent workers.
[Parallel(n_jobs=32)]: Done  88 out of 100 | elapsed:    0.6s remaining:    0.1s
[Parallel(n_jobs=32)]: Done 100 out of 100 | elapsed:    0.7s finished


New sensible chunk: 332127


[Parallel(n_jobs=32)]: Using backend ThreadingBackend with 32 concurrent workers.
[Parallel(n_jobs=32)]: Done  88 out of 100 | elapsed:    1.3s remaining:    0.2s
[Parallel(n_jobs=32)]: Done 100 out of 100 | elapsed:    1.5s finished


Disaggregation is done..
============ Relative error in total energy: 0.009994332164012748
============ Mean absolute error(in Watts): 3.714192880446433
============ Recall: 0.7119340811926773
============ Precision: 0.688852354688163
============ Accuracy: 0.9613564166437941
============ F1 Score: 0.7002030511581103
********************
[GBM-microwave]
********************
train on chunk is done..
elapsed time (secs) = 77.44414496421814
New sensible chunk: 357621
New sensible chunk: 173735
New sensible chunk: 332127
Disaggregation is done..
============ Relative error in total energy: 0.011029575936066158
============ Mean absolute error(in Watts): 9.04897982028675
============ Recall: 0.1237073848065188
============ Precision: 0.26535251009131167
============ Accuracy: 0.9227441020705724
============ F1 Score: 0.16874556080298067
********************
[RESNET-microwave]
********************
16/16 [==============================] - 39s 2s/step - loss: 1.1665 - mean_absolute_error: 0.49